In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

## 使用 MNIST 資料集  (載入資料)

In [2]:
train_dataset = dsets.MNIST(
    root = './data',
    train = True,
    transform = transforms.ToTensor(),
    download = True
)
test_dataset = dsets.MNIST(
    root = './data',
    train = False,
    transform = transforms.ToTensor()
)

In [4]:
print('train_dataset :', train_dataset.train_data.size())

train_dataset : torch.Size([60000, 28, 28])


In [6]:
print('train_dataset :', train_dataset.train_labels.size())

train_dataset : torch.Size([60000])


In [7]:
print('test_dataset :', test_dataset.test_data.size())

test_dataset : torch.Size([10000, 28, 28])


In [9]:
print('test_dataset :', test_dataset.test_labels.size())

test_dataset : torch.Size([10000])


In [12]:
batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = False)

## Create Model !

In [19]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(RNNModel, self).__init__()
        self.hiddens = 100
        self.layers = 1
        
        self.rnn = nn.RNN(input_dim, self.hiddens, self.layers, batch_first=True, nonlinearity='relu')
        self.fc = nn.Linear(self.hiddens, output_dim)
        
    def forward(self, x):
        h0 = Variable(torch.zeros(self.layers, x.size(0), self.hiddens))
        out, hn = self.rnn(x, h0)
        
        out = self.fc(out[:, -1, :])
        return out

In [20]:
input_dim = 28
output_dim = 10
model = RNNModel(input_dim, output_dim)

In [23]:
cost_fun = nn.CrossEntropyLoss()

In [22]:
lr = 0.1
opt = torch.optim.SGD(model.parameters(), lr = lr)

In [26]:
iters = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28, 28))
        labels = Variable(labels)
        
        opt.zero_grad()
        
        outputs = model(images)
        
        loss = cost_fun(outputs, labels)
        loss.backward()
        opt.step()
        
        iters += 1
        
        if iters % 500 == 0:
            correct = 0
            total = 0
            for images, labels in test_loader:
                images = Variable(images.view(-1, 28, 28))
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                
                correct += (predicted == labels).sum()
            acc = 100 * correct / total
            print('Iteration: {}. Loss: {}. Acc: {}'.format(iters, loss.data[0], acc))
                

Iteration: 500. Loss: 0.8604956269264221. Acc: 68.82
Iteration: 1000. Loss: 0.5679857134819031. Acc: 87.13
Iteration: 1500. Loss: 0.28016459941864014. Acc: 91.2
Iteration: 2000. Loss: 0.33720892667770386. Acc: 91.27
Iteration: 2500. Loss: 0.3381904661655426. Acc: 92.22
Iteration: 3000. Loss: 0.3617628812789917. Acc: 90.88
